In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
openai_api_key="your OPENAI API KEY"

This notebook will be broken down into first benchmarking Vanilla GPT models, then GPT+RAG, then Mistral and mistral fine tuned, and finally Biomistral and Biomistral finetuned.

#Vanilla GPT Models

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.0/311.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI

client = OpenAI(api_key=openai_api_key)


In [ ]:
import pandas as pd
df_questions=pd.read_csv('/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Clean Data/Paper 1/p1_400q_eval_ds.csv')

In [ ]:
def answer(evaluator_answer,model_):
    # Define the new task for the assistant
    prompt = (
        f"""Evaluator: {evaluator_answer} \n\n
        Classify the evaluator's evaluation into whether Teacher 1 was better, tied or worse according to the evaluator. If it was better Answer with class '2',
         class '1' if it is a tie,
         class '0' if it's worse. Answer:"""
    )

    # Update system message to match the new task
    system_message = "You are an assistant that extracts whether acording to an answer, you classify whether Teacher 1 was better, tied, or was worse than teacher 2."

    # Use OpenAI API to generate the response
    response = client.chat.completions.create(
        model=model_,  # Adjust the model as per availability or requirements
        messages=[
            {
                "role": "system",
                "content": system_message
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=0.3,  # Adjust temperature as needed to balance creativity and accuracy
        max_tokens=20,  # Adjust max tokens if more are needed for longer answers
        top_p=1
    )

    complete_response = response.choices[0].message.content.strip()
    # Return the generated response
    return complete_response.strip()


In [ ]:
answer(df_questions['Formatted Question'][0],'gpt-4',3)

'D'

In [ ]:
from tqdm import tqdm
answer_gpt4=[answer(question,'gpt-4',3) for question in tqdm(df_questions['Formatted Question'], position=0, leave=True)]
df_questions['gpt-4 Answer']=answer_gpt4


100%|██████████| 452/452 [04:33<00:00,  1.65it/s]


In [ ]:
from tqdm import tqdm
answer_gpt3=[answer(question,'gpt-3.5-turbo',3) for question in tqdm(df_questions['Formatted Question'], position=0, leave=True)]
df_questions['gpt-3.5-turbo Answer']=answer_gpt3

100%|██████████| 452/452 [02:56<00:00,  2.55it/s]


In [ ]:
df_questions

,Raw question,Formatted Question and Markscheme,Formatted Question,Markscheme,gpt-4 Answer,gpt-3.5-turbo Answer
0,\n\nWhat is an example of the therapeutic use ...,Question: What is an example of the therapeuti...,What is an example of the therapeutic use of s...,D,D,D
1,\n\nWhich substance is used for structure in p...,Question: Which substance is used for structur...,Which substance is used for structure in plant...,B,B,B
2,\n\nAn unknown cell is observed using a micros...,Question: An unknown cell is observed using a ...,An unknown cell is observed using a microscope...,C,C,C.
3,\n\nWhat is proportional to a cell’s surface a...,Question: What is proportional to a cell’s sur...,What is proportional to a cell’s surface area?...,A,A,A
4,\n\nWhat are stem cells?\n\nA. Specialized cel...,Question: What are stem cells?\n\nA. Specializ...,What are stem cells?\n\nA. Specialized cells t...,C,C,C
...,...,...,...,...,...,...
447,\n\nWhen a pathogen is ingested by a phagocyte...,Question: When a pathogen is ingested by a pha...,"When a pathogen is ingested by a phagocyte, wh...",C,C,C.
448,\n\nWhat is the role of HCG (human chorionic g...,Question: What is the role of HCG (human chori...,What is the role of HCG (human chorionic gonad...,B,B,B.
449,\n\nWhat is the main role of nerves in human m...,Question: What is the main role of nerves in h...,What is the main role of nerves in human movem...,D,D,D
450,\n\nThrough what process does a spermatid beco...,Question: Through what process does a spermati...,Through what process does a spermatid become a...,B,B,B.


In [ ]:
import re

def extract_first_option(string):
    # Use regular expression to find the first occurrence of A, B, C, or D
    match = re.search(r'\b[A-D]\b', string)
    if match:
        return match.group()
    else:
        # If no match is found, you might want to return a default value or handle it differently
        return '###'

# Assuming answer_gpt3 is  list of strings from GPT-3
list_just_answer_gpt3 = [extract_first_option(string) for string in answer_gpt3]
list_just_answer_gpt4 = [extract_first_option(string) for string in answer_gpt4]

df_questions['gpt-3.5-turbo Answer']=list_just_answer_gpt3
df_questions['gpt-4 Answer']=list_just_answer_gpt4

In [ ]:
df_questions

,Raw question,Formatted Question and Markscheme,Formatted Question,Markscheme,gpt-4 Answer,gpt-3.5-turbo Answer
0,\n\nWhat is an example of the therapeutic use ...,Question: What is an example of the therapeuti...,What is an example of the therapeutic use of s...,D,D,D
1,\n\nWhich substance is used for structure in p...,Question: Which substance is used for structur...,Which substance is used for structure in plant...,B,B,B
2,\n\nAn unknown cell is observed using a micros...,Question: An unknown cell is observed using a ...,An unknown cell is observed using a microscope...,C,C,C
3,\n\nWhat is proportional to a cell’s surface a...,Question: What is proportional to a cell’s sur...,What is proportional to a cell’s surface area?...,A,A,A
4,\n\nWhat are stem cells?\n\nA. Specialized cel...,Question: What are stem cells?\n\nA. Specializ...,What are stem cells?\n\nA. Specialized cells t...,C,C,C
...,...,...,...,...,...,...
447,\n\nWhen a pathogen is ingested by a phagocyte...,Question: When a pathogen is ingested by a pha...,"When a pathogen is ingested by a phagocyte, wh...",C,C,C
448,\n\nWhat is the role of HCG (human chorionic g...,Question: What is the role of HCG (human chori...,What is the role of HCG (human chorionic gonad...,B,B,B
449,\n\nWhat is the main role of nerves in human m...,Question: What is the main role of nerves in h...,What is the main role of nerves in human movem...,D,D,D
450,\n\nThrough what process does a spermatid beco...,Question: Through what process does a spermati...,Through what process does a spermatid become a...,B,B,B


In [ ]:


# Count correct answers given by GPT-4
correct_gpt4 = (df_questions['gpt-4 Answer'] == df_questions['Markscheme']).sum()


# Count correct answers given by GPT-3.5
correct_gpt35 = (df_questions['gpt-3.5-turbo Answer'] == df_questions['Markscheme']).sum()

# Calculate the total number of questions
total_questions = len(df_questions)

# Calculate percentages
percentage_gpt4 = (correct_gpt4 / total_questions) * 100
percentage_gpt35 = (correct_gpt35 / total_questions) * 100

# Print the results
print(f"GPT-4 Correctness Percentage: {percentage_gpt4}%")
print(f"GPT-3.5 Correctness Percentage: {percentage_gpt35}%")


GPT-4 Correctness Percentage: 88.49557522123894%
GPT-3.5 Correctness Percentage: 75.66371681415929%


In [ ]:
df_eval=df_questions
df_eval.to_csv('/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Results/Paper 1/p1_400q_eval_results.csv',index=False)

#Vanilla Mistral Model

In [ ]:
# You only need to run this once per machine
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install --upgrade huggingface_hub
from huggingface_hub import login
login()

In [ ]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [ ]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:


eval_prompt_3="""Question:What is an example of the therapeutic use of stem cells?

A. Sequencing the human genome
B. Forensic investigations of paternity
C. Production of genetically modified crops
D. Restoration of insulation tissue in neurons

Answer:""" #D

In [ ]:
# Init an eval tokenizer that doesn't add padding or eos token
eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
)

model_input_3 = eval_tokenizer(eval_prompt_3, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(f"Eval 3:\n\n")
    print(eval_tokenizer.decode(model.generate(**model_input_3, max_new_tokens=1, repetition_penalty=1.15)[0], skip_special_tokens=True))
    print(f"\n\n")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Eval 3:


Question:What is an example of the therapeutic use of stem cells?

A. Sequencing the human genome
B. Forensic investigations of paternity
C. Production of genetically modified crops
D. Restoration of insulation tissue in neurons

Answer: D





In [ ]:
import pandas as pd
df_eval= pd.read_csv('/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Results/Paper 1/p1_400q_eval_results.csv')
df_eval['Formatted Question'][0]

'What is an example of the therapeutic use of stem cells?\n\nA. Sequencing the human genome \nB. Forensic investigations of paternity \nC. Production of genetically modified crops \nD. Restoration of insulation tissue in neurons'

In [ ]:
def answer_vanilla_mistral(question,tokens):
    question=question+"\n\nRespond with JUST the correct letter. Answer:"
    model_input = eval_tokenizer(question, return_tensors="pt").to("cuda")
    model.eval()
    with torch.no_grad():
        answer=eval_tokenizer.decode(model.generate(**model_input, max_new_tokens=tokens, repetition_penalty=1.15, pad_token_id=eval_tokenizer.eos_token_id)[0], skip_special_tokens=True)
    idx=len(question)+1
    answer=answer[idx:]
    #give me everything after question from my answer string
    return answer

In [ ]:
answer_vanilla_mistral(df_eval['Formatted Question'][0],3)

'D\n\n'

In [ ]:
from tqdm import tqdm
answers_vanilla_mistral=[answer_vanilla_mistral((question),10) for question in tqdm(df_eval['Formatted Question'], position=0, leave=True)]
df_eval['Vanilla Mistral Answer']=answers_vanilla_mistral


100%|██████████| 452/452 [04:22<00:00,  1.72it/s]


In [ ]:
import re

def extract_first_option(string):
    # Use regular expression to find the first occurrence of A, B, C, or D
    match = re.search(r'\b[A-D]\b', string)
    if match:
        return match.group()
    else:
        # If no match return a default value or handle it differently
        return '###'


list_just_answer_vanilla_mistral = [extract_first_option(string) for string in answers_vanilla_mistral]

df_eval['Vanilla Mistral Answer']=list_just_answer_vanilla_mistral

In [ ]:

correct_mistralvanilla = (df_eval['Vanilla Mistral Answer'] == df_eval['Markscheme']).sum()

# Calculate the total number of questions
total_questions = len(df_eval)

# Calculate percentages
percentage_mistralvanilla = (correct_mistralvanilla / total_questions) * 100

# Print the results
print(f"Mistral Vanilla Correctness Percentage: {percentage_mistralvanilla}%")


Mistral Vanilla Correctness Percentage: 55.97345132743363%


In [ ]:
df_eval

In [ ]:
df_eval.to_csv('/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Results/Paper 1/p1_400q_eval_results.csv',index=False)

#Fine Tuned Mistral

In [ ]:
# You only need to run this once per machine
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git #This i can probably remove


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install --upgrade huggingface_hub
from huggingface_hub import login
login()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
)

eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/Arunadiss/main/Finetune/model/mistral/logs/mistral-v3-ib-bio-v1/checkpoint-1500")

In [ ]:
eval_prompt_3="""Question:What is an example of the therapeutic use of stem cells?

A. Sequencing the human genome
B. Forensic investigations of paternity
C. Production of genetically modified crops
D. Restoration of insulation tissue in neurons

Answer:""" #D

In [ ]:
# Init an eval tokenizer that doesn't add padding or eos token
eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
)

model_input_3 = eval_tokenizer(eval_prompt_3, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(f"Eval 3:\n\n")
    print(eval_tokenizer.decode(ft_model.generate(**model_input_3, max_new_tokens=10, repetition_penalty=1.15)[0], skip_special_tokens=True))
    print(f"\n##\n")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Eval 3:


Question:What is an example of the therapeutic use of stem cells?

A. Sequencing the human genome
B. Forensic investigations of paternity
C. Production of genetically modified crops
D. Restoration of insulation tissue in neurons

Answer: D

Explanation: Stem cells

##



In [ ]:
import pandas as pd
df_eval= pd.read_csv('/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Results/Paper 1/p1_400q_eval_results.csv')
df_eval['Formatted Question'][0]

'What is an example of the therapeutic use of stem cells?\n\nA. Sequencing the human genome \nB. Forensic investigations of paternity \nC. Production of genetically modified crops \nD. Restoration of insulation tissue in neurons'

In [ ]:
def answer_ft_mistral(question,tokens):
    question=question+'\n\nRespond with JUST the correct letter. Answer:'
    model_input = eval_tokenizer(question, return_tensors="pt").to("cuda")
    ft_model.eval()
    with torch.no_grad():
        answer=eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=tokens, repetition_penalty=1.15, pad_token_id=eval_tokenizer.eos_token_id)[0], skip_special_tokens=True)
    idx=len(question)+1
    answer=answer[idx:]
    #give me everything after question from my answer string
    return answer

In [ ]:
answer_ft_mistral(df_eval['Formatted Question'][0],10)

'D\n\nWhat are the two main types of'

In [ ]:
from tqdm import tqdm
answers_ft_mistral=[answer_ft_mistral(question,10) for question in tqdm(df_eval['Formatted Question'], position=0, leave=True)]

df_eval['Fine Tuned Mistral Answer']=answers_ft_mistral

100%|██████████| 452/452 [07:10<00:00,  1.05it/s]


In [ ]:
df_eval['Fine Tuned Mistral Answer']=answers_ft_mistral

In [ ]:
import re

def extract_first_option(string):
    # Use regular expression to find the first occurrence of A, B, C, or D
    match = re.search(r'\b[A-D]\b', string)
    if match:
        return match.group()
    else:
        # If no match return a default value or handle it differently
        return '###'
list_just_answers_ft_mistral = [extract_first_option(string) for string in answers_ft_mistral]

df_eval['Fine Tuned Mistral Answer']=list_just_answers_ft_mistral

In [ ]:

correct_mistralft= (df_eval['Fine Tuned Mistral Answer'] == df_eval['Markscheme']).sum()

# Calculate the total number of questions
total_questions = len(df_eval)

# Calculate percentages
percentage_mistralft= (correct_mistralft / total_questions) * 100

# Print the results
print(f"Mistral Fine Tuned Correctness Percentage: {percentage_mistralft}%")


Mistral Fine Tuned Correctness Percentage: 55.97345132743363%


In [ ]:
df_eval

,Raw question,Formatted Question and Markscheme,Formatted Question,Markscheme,gpt-4 Answer,gpt-3.5-turbo Answer,RAG gpt-4 Answer,RAG gpt-3.5-turbo Answer,Vanilla Mistral Answer,Fine Tuned Mistral Answer,Vanilla Biomistral DARE Answer,Fine Tuned Biomistral DARE Answer
0,\n\nWhat is an example of the therapeutic use ...,Question: What is an example of the therapeuti...,What is an example of the therapeutic use of s...,D,D,D,D,D,D,D,D,D
1,\n\nWhich substance is used for structure in p...,Question: Which substance is used for structur...,Which substance is used for structure in plant...,B,B,B,B,B,B,B,B,B
2,\n\nAn unknown cell is observed using a micros...,Question: An unknown cell is observed using a ...,An unknown cell is observed using a microscope...,C,C,C,C,A,C,C,C,C
3,\n\nWhat is proportional to a cell’s surface a...,Question: What is proportional to a cell’s sur...,What is proportional to a cell’s surface area?...,A,A,A,A,A,D,D,A,A
4,\n\nWhat are stem cells?\n\nA. Specialized cel...,Question: What are stem cells?\n\nA. Specializ...,What are stem cells?\n\nA. Specialized cells t...,C,C,C,C,C,A,A,C,C
...,...,...,...,...,...,...,...,...,...,...,...,...
447,\n\nWhen a pathogen is ingested by a phagocyte...,Question: When a pathogen is ingested by a pha...,"When a pathogen is ingested by a phagocyte, wh...",C,C,C,C,C,C,C,C,C
448,\n\nWhat is the role of HCG (human chorionic g...,Question: What is the role of HCG (human chori...,What is the role of HCG (human chorionic gonad...,B,B,B,B,B,B,B,D,D
449,\n\nWhat is the main role of nerves in human m...,Question: What is the main role of nerves in h...,What is the main role of nerves in human movem...,D,D,D,D,D,D,D,D,D
450,\n\nThrough what process does a spermatid beco...,Question: Through what process does a spermati...,Through what process does a spermatid become a...,B,B,B,B,B,B,B,D,B


In [ ]:
df_eval.to_csv('/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Results/Paper 1/p1_400q_eval_results.csv',index=False)

#RAG GPT

In [ ]:
# Check your Colab temporary path
!pwd # Returns /content

# Copy custom module from Google Drive to Colab temporary drive
! cp /content/drive/MyDrive/Arunadiss/main/RAG/v0/rag_models.py /content/rag_models.py

/content


In [ ]:
!pip install langchain
!pip install openai
!pip install faiss-cpu
!pip install langchain_experimental
#!pip install "langchain[docarray]"
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.8 MB/s eta 0:00:00


##Example Inference

In [ ]:
from rag_models import Rag
from langchain_community.embeddings.openai import OpenAIEmbeddings


In [ ]:
folder_path = "/content/drive/MyDrive/Arunadiss/main/RAG/v0/vector_store"
rag_instance = Rag(folder_path,openai_api_key,OpenAIEmbeddings(openai_api_key=openai_api_key))
question = "What is DNA Replication?"
response = rag_instance.generate_response(question,expert_tutor=True,n_tokens=500, model="gpt-4")
print("Response:", response[0])
print("/n/n Time taken", response[3])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Response: DNA replication is a semi-conservative process where the original (parental) DNA molecule serves as a template for the production of a new (daughter) DNA molecule. It involves a complex system of enzymes, each with a specific role.

The process begins with the enzyme helicase, which unwinds and separates the double-stranded DNA by breaking hydrogen bonds between the base pairs. This creates a 'replication fork' of two strands running in opposite directions. DNA gyrase reduces the torsional strain created by this unwinding.

After the strands are separated, Single Stranded Binding (SSB) proteins bind to them, preventing them from re-annealing and protecting them from being digested by nucleases. Then, DNA primase generates a short RNA primer on each of the template strands, providing an initiation point for DNA polymerase III.

DNA polymerase III attaches to the 3’-end of the primer and joins the free nucleotides together in a 5’ → 3’ direction. On the leading strand, DNA poly

In [ ]:
question = "What is DNA Replication?"
response = rag_instance.generate_response(question,expert_tutor=False,n_tokens=500, model="gpt-4")
print("Response:", response[0])
print("/n/n Time taken", response[3])

Response: DNA replication is a biological process by which a DNA molecule is copied to produce two identical DNA molecules. It is vital for cell division, growth, and repair. In this process, the double helix of the existing DNA molecule separates into two strands. Each of these strands then serves as a template for the formation of a complementary strand, resulting in two identical copies of the original DNA molecule.
/n/n Time taken 5.27


In [ ]:
question = "What is DNA Replication?"
response = rag_instance.generate_response(question,expert_tutor=True,n_tokens=500, model="gpt-3.5-turbo")
print("Response:", response[0])
print("/n/n Time taken", response[3])

Response: DNA replication is a fundamental biological process where a cell creates an identical copy of its DNA. This process is crucial for cell division, growth, and repair. During DNA replication, a complex system of enzymes works together to unwind the double-stranded DNA molecule. Helicase plays a key role in unwinding and separating the two strands by breaking the hydrogen bonds between the base pairs, creating a replication fork.

After the strands are separated, DNA gyrase reduces the torsional strain caused by unwinding, while Single-Stranded Binding (SSB) Proteins prevent the strands from re-annealing and being digested. DNA Primase generates RNA primers on the template strands, providing a starting point for DNA Polymerase III to extend the nucleotide chain. DNA Polymerase III attaches to the primer and joins free nucleotides together in a 5’ → 3’ direction.

DNA replication occurs in opposite directions on the two strands due to their antiparallel nature. The leading strand

In [ ]:
question = "What is DNA Replication?"
response = rag_instance.generate_response(question,expert_tutor=False,n_tokens=500, model="gpt-3.5-turbo")
print("Response:", response[0])
print("/n/n Time taken", response[3])

Response: DNA replication is the process by which a cell makes an identical copy of its DNA. This essential process occurs before cell division to ensure that each daughter cell receives a complete set of genetic information. During DNA replication, the double-stranded DNA molecule unwinds and each strand serves as a template for the synthesis of a new complementary strand. This results in two identical DNA molecules, each containing one original strand and one newly synthesized strand.
/n/n Time taken 2.67


##Evaluation

In [ ]:
import pandas as pd
df_eval= pd.read_csv('/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Results/Paper 1/p1_400q_eval_results.csv')
df_eval['Formatted Question'][0]

'What is an example of the therapeutic use of stem cells?\n\nA. Sequencing the human genome \nB. Forensic investigations of paternity \nC. Production of genetically modified crops \nD. Restoration of insulation tissue in neurons'

In [ ]:
from rag_models import Rag
from openai import OpenAI
from langchain_community.embeddings.openai import OpenAIEmbeddings
folder_path = "/content/drive/MyDrive/Arunadiss/main/RAG/v0/vector_store"
rag_instance = Rag(folder_path,openai_api_key,OpenAIEmbeddings(openai_api_key=openai_api_key))


In [ ]:
question=df_eval['Formatted Question'][90]+'/n/n Only respond with the correct letter!'
print(rag_instance.generate_response(question,expert_tutor=True,n_tokens=3, model="gpt-4")[0])

A


In [ ]:
from tqdm import tqdm
answers_rag_gpt4=[rag_instance.generate_response((question+'/n/n Only respond with the correct letter!'),expert_tutor=True,n_tokens=3, model="gpt-4")[0] for question in tqdm(df_eval['Formatted Question'], position=0, leave=True)]
df_eval['RAG gpt-4 Answer']=answers_rag_gpt4


100%|██████████| 452/452 [06:22<00:00,  1.18it/s]


In [ ]:
from tqdm import tqdm
answers_rag_gpt35=[rag_instance.generate_response((question+'/n/n Only respond with the correct letter!'),expert_tutor=True,n_tokens=3, model="gpt-3.5-turbo")[0] for question in tqdm(df_eval['Formatted Question'], position=0, leave=True)]
df_eval['RAG gpt-3.5-turbo Answer']=answers_rag_gpt35


100%|██████████| 452/452 [09:11<00:00,  1.22s/it]


In [ ]:
import re

def extract_first_option(string):
    # Use regular expression to find the first occurrence of A, B, C, or D
    match = re.search(r'\b[A-D]\b', string)
    if match:
        return match.group()
    else:
        return '###'

list_just_answer_rag_gpt35 = [extract_first_option(string) for string in answers_rag_gpt35]
df_eval['RAG gpt-3.5-turbo Answer']=list_just_answer_rag_gpt35
list_just_answer_rag_gpt4 = [extract_first_option(string) for string in answers_rag_gpt4]
df_eval['RAG gpt-4 Answer']=list_just_answer_rag_gpt4

In [ ]:
# Count correct answers given by RAG GPT-3.5
correct_raggpt4= (df_eval['RAG gpt-4 Answer'] == df_eval['Markscheme']).sum()

# Count correct answers given by RAG GPT-3.5
correct_raggpt35 = (df_eval['RAG gpt-3.5-turbo Answer'] == df_eval['Markscheme']).sum()

# Calculate the total number of questions
total_questions = len(df_eval)

# Calculate percentages
percentage_raggpt4 = (correct_raggpt4 / total_questions) * 100

percentage_raggpt35 = (correct_raggpt35 / total_questions) * 100

# Print the results
print(f"RAG GPT 4 Correctness Percentage: {percentage_raggpt4}%")

print(f"RAG GPT 3.5-turbo Correctness Percentage: {percentage_raggpt35}%")


RAG GPT 4 Correctness Percentage: 90.48672566371681%
RAG GPT 3.5-turbo Correctness Percentage: 74.77876106194691%


In [ ]:
df_eval

,Raw question,Formatted Question and Markscheme,Formatted Question,Markscheme,gpt-4 Answer,gpt-3.5-turbo Answer,RAG gpt-4 Answer,RAG gpt-3.5-turbo Answer
0,\n\nWhat is an example of the therapeutic use ...,Question: What is an example of the therapeuti...,What is an example of the therapeutic use of s...,D,D,D,D,D
1,\n\nWhich substance is used for structure in p...,Question: Which substance is used for structur...,Which substance is used for structure in plant...,B,B,B,B,B
2,\n\nAn unknown cell is observed using a micros...,Question: An unknown cell is observed using a ...,An unknown cell is observed using a microscope...,C,C,C,C,A
3,\n\nWhat is proportional to a cell’s surface a...,Question: What is proportional to a cell’s sur...,What is proportional to a cell’s surface area?...,A,A,A,A,A
4,\n\nWhat are stem cells?\n\nA. Specialized cel...,Question: What are stem cells?\n\nA. Specializ...,What are stem cells?\n\nA. Specialized cells t...,C,C,C,C,C
...,...,...,...,...,...,...,...,...
447,\n\nWhen a pathogen is ingested by a phagocyte...,Question: When a pathogen is ingested by a pha...,"When a pathogen is ingested by a phagocyte, wh...",C,C,C,C,C
448,\n\nWhat is the role of HCG (human chorionic g...,Question: What is the role of HCG (human chori...,What is the role of HCG (human chorionic gonad...,B,B,B,B,B
449,\n\nWhat is the main role of nerves in human m...,Question: What is the main role of nerves in h...,What is the main role of nerves in human movem...,D,D,D,D,D
450,\n\nThrough what process does a spermatid beco...,Question: Through what process does a spermati...,Through what process does a spermatid become a...,B,B,B,B,B


In [ ]:
df_eval.to_csv('/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Results/Paper 1/p1_400q_eval_results.csv',index=False)

#Vanilla BioMistral-7B-DARE

In [ ]:
# You only need to run this once per machine
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 102.2/102.2 MB 198.2 MB/s eta 0:00:01^C
Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 8, in <module>
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 77, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/__init__.py", line 114, in create_command
    module = importlib.import_module(module_path)
  File "/usr/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module

In [ ]:
!pip install --upgrade huggingface_hub
from huggingface_hub import login
login()

In [ ]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "BioMistral/BioMistral-7B-DARE"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="auto")

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/22.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:

eval_prompt_3="""Question:What is an example of the therapeutic use of stem cells?

A. Sequencing the human genome
B. Forensic investigations of paternity
C. Production of genetically modified crops
D. Restoration of insulation tissue in neurons

Answer:""" #D

In [ ]:
#Apply accelerator
model = accelerator.prepare_model(model)
# Init an eval tokenizer that doesn't add padding or eos token
eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
)

model_input_3 = eval_tokenizer(eval_prompt_3, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(f"Eval 3:\n\n")
    print(eval_tokenizer.decode(model.generate(**model_input_3, max_new_tokens=1, repetition_penalty=1.15)[0], skip_special_tokens=True))
    print(f"\n\n")


tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Eval 3:


Question:What is an example of the therapeutic use of stem cells?

A. Sequencing the human genome
B. Forensic investigations of paternity
C. Production of genetically modified crops
D. Restoration of insulation tissue in neurons

Answer: D





In [ ]:
import pandas as pd
df_eval= pd.read_csv('/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Results/Paper 1/p1_400q_eval_results.csv')
df_eval['Formatted Question'][0]

'What is an example of the therapeutic use of stem cells?\n\nA. Sequencing the human genome \nB. Forensic investigations of paternity \nC. Production of genetically modified crops \nD. Restoration of insulation tissue in neurons'

In [ ]:
def answer_vanilla_biomistral(question,tokens):
    question=question+'\n\nRespond with JUST the correct letter. Answer:'
    model_input = eval_tokenizer(question, return_tensors="pt").to("cuda")
    model.eval()
    with torch.no_grad():
        answer=eval_tokenizer.decode(model.generate(**model_input, max_new_tokens=tokens, repetition_penalty=1.15, pad_token_id=eval_tokenizer.eos_token_id)[0], skip_special_tokens=True)
    idx=len(question)+1
    answer=answer[idx:]
    #give me everything after question from my answer string
    return answer

In [ ]:
answer_vanilla_biomistral(df_eval['Formatted Question'][0],1)

'D'

In [ ]:
from tqdm import tqdm
answers_vanilla_biomistral=[answer_vanilla_biomistral(question,10) for question in tqdm(df_eval['Formatted Question'], position=0, leave=True)]
df_eval['Vanilla Biomistral DARE Answer']=answers_vanilla_biomistral


100%|██████████| 452/452 [01:34<00:00,  4.78it/s]


In [ ]:
import re

def extract_first_option(string):
    # Use regular expression to find the first occurrence of A, B, C, or D
    match = re.search(r'\b[A-D]\b', string)
    if match:
        return match.group()
    else:
        # If no match return a default value or handle it differently
        return '###'


list_just_answer_vanilla_biomistral = [extract_first_option(string) for string in answers_vanilla_biomistral]

df_eval['Vanilla Biomistral DARE Answer']=list_just_answer_vanilla_biomistral

In [ ]:

correct_biomistralvanilla = (df_eval['Vanilla Biomistral DARE Answer'] == df_eval['Markscheme']).sum()

# Calculate the total number of questions
total_questions = len(df_eval)

# Calculate percentages
percentage_biomistralvanilla = (correct_biomistralvanilla / total_questions) * 100

# Print the results
print(f"Vanilla Biomistral DARE Correctness Percentage: {percentage_biomistralvanilla}%")


Vanilla Biomistral DARE Correctness Percentage: 58.62831858407079%


In [ ]:
df_eval

,Raw question,Formatted Question and Markscheme,Formatted Question,Markscheme,gpt-4 Answer,gpt-3.5-turbo Answer,RAG gpt-4 Answer,RAG gpt-3.5-turbo Answer,Vanilla Mistral Answer,Fine Tuned Mistral Answer,Vanilla Biomistral DARE Answer
0,\n\nWhat is an example of the therapeutic use ...,Question: What is an example of the therapeuti...,What is an example of the therapeutic use of s...,D,D,D,D,D,D,D,D
1,\n\nWhich substance is used for structure in p...,Question: Which substance is used for structur...,Which substance is used for structure in plant...,B,B,B,B,B,B,B,B
2,\n\nAn unknown cell is observed using a micros...,Question: An unknown cell is observed using a ...,An unknown cell is observed using a microscope...,C,C,C,C,A,C,C,C
3,\n\nWhat is proportional to a cell’s surface a...,Question: What is proportional to a cell’s sur...,What is proportional to a cell’s surface area?...,A,A,A,A,A,D,D,A
4,\n\nWhat are stem cells?\n\nA. Specialized cel...,Question: What are stem cells?\n\nA. Specializ...,What are stem cells?\n\nA. Specialized cells t...,C,C,C,C,C,A,A,C
...,...,...,...,...,...,...,...,...,...,...,...
447,\n\nWhen a pathogen is ingested by a phagocyte...,Question: When a pathogen is ingested by a pha...,"When a pathogen is ingested by a phagocyte, wh...",C,C,C,C,C,C,C,C
448,\n\nWhat is the role of HCG (human chorionic g...,Question: What is the role of HCG (human chori...,What is the role of HCG (human chorionic gonad...,B,B,B,B,B,B,B,D
449,\n\nWhat is the main role of nerves in human m...,Question: What is the main role of nerves in h...,What is the main role of nerves in human movem...,D,D,D,D,D,D,D,D
450,\n\nThrough what process does a spermatid beco...,Question: Through what process does a spermati...,Through what process does a spermatid become a...,B,B,B,B,B,B,B,D


In [ ]:
df_eval.to_csv('/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Results/Paper 1/p1_400q_eval_results.csv',index=False)

#Fine Tuned Biomistral DARE

In [ ]:
# You only need to run this once per machine
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 90.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.0 MB/s eta 0:00:00
  Installing bui

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "BioMistral/BioMistral-7B-DARE"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
)

eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, '/content/drive/MyDrive/Arunadiss/main/Finetune/model/biomistral-dare/logs/biomistral-dare-v3-ib-bio-v1/checkpoint-1500')

In [ ]:
eval_prompt_3="""Question:What is an example of the therapeutic use of stem cells?

A. Sequencing the human genome
B. Forensic investigations of paternity
C. Production of genetically modified crops
D. Restoration of insulation tissue in neurons

Answer:""" #D

In [ ]:
ft_model = accelerator.prepare_model(ft_model)
# Init an eval tokenizer that doesn't add padding or eos token
eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
)

model_input_3 = eval_tokenizer(eval_prompt_3, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(f"Eval 3:\n\n")
    print(eval_tokenizer.decode(ft_model.generate(**model_input_3, max_new_tokens=5, repetition_penalty=1.15)[0], skip_special_tokens=True))
    print(f"\n##\n")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Eval 3:


Question:What is an example of the therapeutic use of stem cells?

A. Sequencing the human genome
B. Forensic investigations of paternity
C. Production of genetically modified crops
D. Restoration of insulation tissue in neurons

Answer: D. Restoration of

##



In [ ]:
import pandas as pd
df_eval= pd.read_csv('/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Results/Paper 1/p1_400q_eval_results.csv')
df_eval['Formatted Question'][0]

'What is an example of the therapeutic use of stem cells?\n\nA. Sequencing the human genome \nB. Forensic investigations of paternity \nC. Production of genetically modified crops \nD. Restoration of insulation tissue in neurons'

In [ ]:
def answer_ft_mistral(question,tokens):
    question=question+"\n\nRespond with JUST the correct letter. Answer:"#'/nRespond with the letter that correctly answers the question \n Answer:'
    model_input = eval_tokenizer(question, return_tensors="pt").to("cuda")
    ft_model.eval()
    with torch.no_grad():
        answer=eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=tokens, repetition_penalty=1.15, pad_token_id=eval_tokenizer.eos_token_id)[0], skip_special_tokens=True)
    idx=len(question)+1
    answer=answer[idx:]
    #give me everything after question from my answer string
    return answer

In [ ]:
answer_ft_mistral(df_eval['Formatted Question'][0],10)

'D.'

In [ ]:
from tqdm import tqdm
answers_ft_biomistral=[answer_ft_mistral(question,5) for question in tqdm(df_eval['Formatted Question'], position=0, leave=True)]



100%|██████████| 452/452 [05:13<00:00,  1.44it/s]


In [ ]:
answers_ft_biomistral

In [ ]:
import re

def extract_first_option(string):
    # Use regular expression to find the first occurrence of A, B, C, or D
    match = re.search(r'\b[A-D]\b', string)
    if match:
        return match.group()
    else:
        # If no match return a default value or handle it differently
        return '###'


list_just_answer_ft_biomistral = [extract_first_option(string) for string in answers_ft_biomistral]

df_eval['Fine Tuned Biomistral DARE Answer']=list_just_answer_ft_biomistral

In [ ]:

correct_ft_biomistral = (df_eval['Fine Tuned Biomistral DARE Answer'] == df_eval['Markscheme']).sum()

# Calculate the total number of questions
total_questions = len(df_eval)

# Calculate percentages
percentage_ft_biomistral = (correct_ft_biomistral / total_questions) * 100

# Print the results
print(f"Fine Tuned Biomistral DARE Correctness Percentage: {percentage_ft_biomistral}%")


Fine Tuned Biomistral DARE Correctness Percentage: 60.61946902654868%


In [ ]:

df_eval

,Raw question,Formatted Question and Markscheme,Formatted Question,Markscheme,gpt-4 Answer,gpt-3.5-turbo Answer,RAG gpt-4 Answer,RAG gpt-3.5-turbo Answer,Vanilla Mistral Answer,Fine Tuned Mistral Answer,Vanilla Biomistral DARE Answer,Fine Tuned Biomistral DARE Answer
0,\n\nWhat is an example of the therapeutic use ...,Question: What is an example of the therapeuti...,What is an example of the therapeutic use of s...,D,D,D,D,D,D,D,D,D
1,\n\nWhich substance is used for structure in p...,Question: Which substance is used for structur...,Which substance is used for structure in plant...,B,B,B,B,B,B,B,B,B
2,\n\nAn unknown cell is observed using a micros...,Question: An unknown cell is observed using a ...,An unknown cell is observed using a microscope...,C,C,C,C,A,C,C,C,C
3,\n\nWhat is proportional to a cell’s surface a...,Question: What is proportional to a cell’s sur...,What is proportional to a cell’s surface area?...,A,A,A,A,A,D,D,A,A
4,\n\nWhat are stem cells?\n\nA. Specialized cel...,Question: What are stem cells?\n\nA. Specializ...,What are stem cells?\n\nA. Specialized cells t...,C,C,C,C,C,A,A,C,C
...,...,...,...,...,...,...,...,...,...,...,...,...
447,\n\nWhen a pathogen is ingested by a phagocyte...,Question: When a pathogen is ingested by a pha...,"When a pathogen is ingested by a phagocyte, wh...",C,C,C,C,C,C,C,C,C
448,\n\nWhat is the role of HCG (human chorionic g...,Question: What is the role of HCG (human chori...,What is the role of HCG (human chorionic gonad...,B,B,B,B,B,B,B,D,D
449,\n\nWhat is the main role of nerves in human m...,Question: What is the main role of nerves in h...,What is the main role of nerves in human movem...,D,D,D,D,D,D,D,D,D
450,\n\nThrough what process does a spermatid beco...,Question: Through what process does a spermati...,Through what process does a spermatid become a...,B,B,B,B,B,B,B,D,B


In [ ]:


df_eval.to_csv('/content/drive/MyDrive/Arunadiss/main/Evaluation/Data/Results/Paper 1/p1_400q_eval_results.csv',index=False)